# Load and Explore Neuroimaging Data with NiBabel in Python

This notebook combines the excellent [Dartbrains Introductory Notebook](https://dartbrains.org/content/Introduction_to_Neuroimaging_Data.html).
and [Christopher J Markiewicz's NeuroHackademy 2020 Notebook](https://nbviewer.org/github/neurohackademy/nh2020-curriculum/blob/master/we-nibabel-markiewicz/NiBabel.ipynb)

**TIP**: Some of these cells are slow to run! To determine whether a cell has been run, look at the square brackets on the left:
- `[ ]` indicates the cell has not been run
- `[*]` indicates the cell is running.  Be patient!
- A number in the brackets, e.g., `[35]`, indicates that the cell has been run, and you can move on to the next one. 

Many Python neuroimaging libraries exist and more are being developed.    
In this notebook, you will explore [NiBabel](https://nipy.org/packages/nilearn/index.html) which reads and writes common neuroimaging file formats.

# Get the Data

Download the data we will be using from [OSF](https://osf.io/5q3m8) with *wget*, unzip it and remove the zip file.

In [ ]:
import os
import wget
import zipfile
import numpy as np

site_url = 'https://osf.io/5q3m8/download'

if (not os.path.isdir('data')):
    wget.download(site_url)
    with zipfile.ZipFile('Jupyter_neuro_data.zip','r') as zip_ref: 
        zip_ref.extractall(path=None)

In [ ]:
if os.path.exists('Jupyter_neuro_data.zip'):
   os.remove('Jupyter_neuro_data.zip')

Now load the data using BIDSLayout from PyBIDS:

In [ ]:
from bids import BIDSLayout, BIDSValidator

data_dir = 'data/inputs'
layout = BIDSLayout(data_dir, derivatives=False)

# Load Data with NiBabel

NiBabel objects can be initialized by specifying a file path to a NIfTI file, and you can find the NIfTI file path using BIDSLayout.  

First, import the NiBabel module as `nib` (a short and sweet alias so that you don't have to type so much when using the tool).     

Our data set has only one T1w image for subject 219, so you can select it using BIDSLayout and load it using NiBabel:

In [ ]:
import nibabel as nib

# Load an anatomical image from subject 219 using PyBIDS. We KNOW we only have one in this example.
file_list = layout.get(subject='219', session='itbs', suffix='T1w', return_type='file', extension='nii.gz')

# Now load the data using NiBabel. Data does NOT have to be in a BIDS data structure to be loaded with NiBabel.
T1w_image = nib.load(file_list[0])  # select the ONE image

## Get Information about the NiBabel Object
When working with NiBabel data objects, you can consult the [online documentation](https://nipy.org/nibabel/tutorials.html#tutorials). More conveniently, you can query a variable by appending a question mark (`?`) to reveal details such as the characteristics of an object and parameters to retrieve more information.

In [ ]:
T1w_image?

### Try This!
Add a cell and append two question marks instead of one. 

Note that this is different from the information that is shown by the Python `print` function:

In [ ]:
print(T1w_image)

### View the Image Header

An image consists of two parts, the data and the header. The header contains image metadata which you can access through the `Nifti1Image` object:

In [ ]:
T1w_image.header

In [ ]:
print(T1w_image.header)

### View the Affine Matrix

The orientation of the image in space is represented in the header as an affine matrix. The affine matrix can be used to transform images between different spaces. View the matrix for our image:

In [ ]:
affine = T1w_image.affine
affine

In [ ]:
print("Orientation:", nib.aff2axcodes(affine))

Convert the center point in world coordinates to voxel coordinates and get intensity at center point:

In [ ]:
i, j, k, _ = np.linalg.pinv(affine) @ [0, 0, 0, 1]
print(f"Center: ({int(i)}, {int(j)}, {int(k)})")
print(f"Value: ", image_data[int(i), int(j), int(k)])

In [ ]:
header = T1w_image.header
header.get_zooms()  # returns voxel sizes

### View the Object Dimensions

The `Nifti1Image` object stores the image data in either a 3D or 4D NumPy array. Just like NumPy, it is easy to get the dimensions and shape of the data using `ndim` and `shape`:

In [ ]:
T1w_image.ndim

In [ ]:
T1w_image.shape

In [ ]:
# several more properties of the internal numpy array are accessible through the dataobj attribute:
dir(T1w_image.dataobj)

## Plot a Slice of the Data Object with matplotlib

As you have seen, this is a 3D image. Directly access the image data and plot a single slice using standard matplotlib functions: 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

image_data = T1w_image.get_fdata()
sslice = image_data[80,:,:]
_ = plt.imshow(sslice, origin='lower')  # origin argument maintains the L-R, P-A origins of this image

To see a view in standard orientation, transpose the slice before plotting it.

In [ ]:
# In this call `_=` prevents the output from listing the class of the object.  
# This just makes the output prettier.

_ = plt.imshow(sslice.transpose([1,0]), origin='lower')

As a shortcut, we can use the `T` attribute of the slice to do the same transpose:

In [ ]:
_ = plt.imshow(sslice.T, origin='lower')

Since we have the 3D image data, we can plot along any axis:

In [ ]:
aslice = image_data[:,:,150]
print(aslice.shape)
_ = plt.imshow(aslice.T, origin='lower')

### Plot Different Slices

For the sagittal view, you retrieved a slice for the first dimension (i.e., [80,:,:].  For the axial view, you retrieved a value for the third dimension [:,:,150]. 

How would you access a coronal view?

### NiBabel OrthoViewer
Instead of using Matplotlib, NiBabel offers a viewer that scales the voxels to reflect their size, includes all three orthogonal views, and labels orientations.

In [ ]:
T1w_image.orthoview()

## Summary of NiBabel

NiBabel is used to load images into Python. Once the images are loaded, you can learn more about them by examining their headers. In addition, you can view and manipulate the images with Python libraries like matplotlib. Let's look at some other Python libraies for neuroimaging.